In [1]:
# !wget https://huggingface.co/datasets/malaysia-ai/fleurs-my-ms/resolve/main/test-fleurs.json
# !wget https://huggingface.co/datasets/malaysia-ai/fleurs-my-ms/resolve/main/fleurs-test.zip
# !unzip fleurs-test.zip

In [2]:
from glob import glob
import json
import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline
from datasets import Audio
from tqdm import tqdm
import jiwer

sr = 16000
audio = Audio(sampling_rate=sr)
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [3]:
processor = AutoProcessor.from_pretrained('openai/whisper-base')
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    'openai/whisper-base',
    use_flash_attention_2 = True,
    torch_dtype = torch.bfloat16
)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


generation_config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

In [4]:
_ = model.cuda()

In [5]:
with open('test-fleurs.json') as fopen:
    data = json.load(fopen)

In [6]:
len(data)

749

In [7]:
wer, cer = [], []

for i in tqdm(range(len(data))):
    y = audio.decode_example(audio.encode_example(data[i]['audio_filename']))['array']
    inputs = processor([y], return_tensors = 'pt', sampling_rate = 16000)
    inputs['input_features'] = inputs['input_features'].type(torch.bfloat16).cuda()
    r = model.generate(inputs['input_features'], language='ms', return_timestamps=True)
    
    out = processor.tokenizer.decode(r[0], skip_special_tokens = True).strip()
    actual = processor.tokenizer.decode(processor.tokenizer.encode(data[i]['new_text']), skip_special_tokens = True).strip()
    
    for p in PUNCTUATION:
        out = out.replace(p, '')
        actual = actual.replace(p, '')
        
    actual = actual.lower()
    out = out.lower()
    
    error = jiwer.wer(actual, out)
    wer.append(error)
    error = jiwer.cer(actual, out)
    cer.append(error)

100%|██████████| 749/749 [08:03<00:00,  1.55it/s]


In [8]:
import numpy as np

np.mean(wer), np.mean(cer)

(0.5164123884823085, 0.24820848114299138)